# k-th Largest Element in Two Sorted Arrays

We are given two sorted arrays in non-decreasing order:
$$
a_1 \le a_2 \le \cdots \le a_n \\
b_1 \le b_2 \le \cdots \le b_n
$$

Our goal is to find the $k$-th largest element in the combined collection
$$
\{a_1, a_2, \ldots, a_n\} \cup \{b_1, b_2, \ldots, b_n\}
$$
in $O(\log k)$ time (assuming $k \le n$).

In [50]:
function random_sorted_vector(n::Int)
    v = rand(n)
    sort!(v)
    return v
end

a = random_sorted_vector(1000)
b = random_sorted_vector(1000)
k = 982;

In [ ]:
using BenchmarkTools

# Add @views so we never copy anything inside the function
@views function largest(a, b, k)
    if k == 1
        return max(a[end], b[end])
    end

    if length(a) > k
        a = a[end - k + 1:end]
    end
    if length(b) > k
        b = b[end - k + 1:end]
    end

    i = min(length(a), k ÷ 2)
    j = min(length(b), k ÷ 2)
    
    # i-th largest el in a
    a_pivot = a[end - i + 1]
    # j-th largest el in b
    b_pivot = b[end - j + 1]

    if a_pivot >= b_pivot
        # we know that elements in a from i to end are larger than the k-th 
        # largest, so we remove them, similarly we can remove part of elements in b because 
        # they are for sure too small 
        return largest(a[1:end - i], b[j:end], k - i)
    else
        # symmetric case 
        return largest(a[i:end], b[1:end - j], k - j)
    end
end


result = @btime largest(a, b, k)
println("Result: ", result)


  131.033 ns (3 allocations: 112 bytes)
Result: 0.4873277029391777


# Linear To Check

In [52]:
using BenchmarkTools

function linear_kth_largest(a, b, k)
    if k == 1
       return max(a[end], b[end]) 
    end

    if a[end] >= b[end]
        return linear_kth_largest(a[1:end-1], b, k-1)
    else
        return linear_kth_largest(a, b[1:end-1], k-1)
    end
end

@btime linear_kth_largest(a, b, k)

  259.083 μs (2944 allocations: 5.76 MiB)


0.4873277029391777

## Check

In [55]:
using Random

function random_sorted_vector(n::Int)
    # Generate n random integers in the range -20 to 20 (with possible duplicates)
    vec = rand(n)
    return sort(vec)
end

l = 10
for _ in 1:200
    a = random_sorted_vector(l)
    b = random_sorted_vector(l)
    k = rand(1:l)

    if linear_kth_largest(a, b,k ) != largest(a,b,k)
        println("INPUT")
        println(a)
        println(b)
        println(k)
        println("OUTPUT")
        println(linear_kth_largest(a, b,k ))
        println(largest(a, b,k ))
        return
    end

end